In [1]:
import os
import cv2
import numpy as np
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
import time
from functools import wraps

In [10]:
def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__} took {total_time:.4f} seconds')
        return result
    return timeit_wrapper

In [18]:
@timeit
def bw(image_path, output_dir):
    # Read image
    image = cv2.imread(image_path)
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_name = "".join(image_path.split('.')[:-1])
    cv2.imwrite(os.path.join(output_dir, f"{image_name}_{bw}.jpg"), gray)
    return gray


In [19]:
@timeit
def blur(image_path, output_dir):
    # Read image
    image = cv2.imread(image_path)
    # Apply blur
    blurred = cv2.GaussianBlur(image, (11, 11), 0)
    image_name = "".join(image_path.split('.')[:-1])
    cv2.imwrite(os.path.join(output_dir, f"{image_name}_{bw}.jpg"), blurred)
    return blurred

In [9]:
@timeit
def noisy(image_path, output_dir):
    # Read image
    image = cv2.imread(image_path)
    # Add noise
    noisy = np.zeros_like(image)
    cv2.randn(noisy, 0, 25)
    noisy_image = cv2.add(image, noisy)
    image_name = "".join(image_path.split('.')[:-1])
    cv2.imwrite(os.path.join(output_dir, f"{image_name}_{bw}.jpg"), noisy_image)
    return noisy_image

NameError: name 'timeit' is not defined

In [ ]:
# find the threshold value using the grayscale image
def find_thresh(grayscale):
    hist = cv2.calcHist([grayscale], [0], None, [256], [0, 256])
    hist_norm = hist/hist.sum()
    black_pixels = 0
    threshold = 0
    bl_fraction = 0
    for i in range(len(hist_norm)):
        bl_fraction += hist_norm[i]
        black_pixels += hist[i]
        if bl_fraction >= 0.5:
            threshold = i
            break
    return threshold, black_pixels[0]


In [ ]:
def salt_pepp_noise(image, noisy_pixels):
    noisy_image = image.copy()
    height, width, _ = noisy_image.shape
    for pixel in range(noisy_pixels):
        row, col = np.random.randint(0, height), np.random.randint(0, width)
        if np.random.rand() < 0.5:
            noisy_image[row, col] = [0, 0, 0]
        else:
            noisy_image[row, col] = [255, 255, 255]
    return noisy_image

In [11]:
@timeit
def transform_image(image_path):
    # Read image
    image = cv2.imread(image_path)
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    (threshold, black_pixels) = find_thresh(gray)
    (thresh, bnw_image) = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    # Apply blur
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    # Add noise
    noisy = np.zeros_like(image)
    cv2.randn(noisy, 0, 25)
    noisy_image = cv2.add(image, noisy)
    return gray, blurred, noisy_image

In [14]:
def transform_images_parallel(input_dir, output_dir):
    image_paths = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.jpg')]
    with Pool(processes=os.cpu_count()) as pool:
        transformed_images = pool.map(transform_image, image_paths)
        
    for i, image_path in enumerate(image_paths):
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        for i, img in enumerate(transformed_images):
            print(type(img))
            print(i)
            cv2.imwrite(os.path.join(output_dir, f"{image_name}_{i}.jpg"), img)

In [15]:
if __name__ == "__main__":
    input_dir = "specify"
    output_dir = "specify"

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    transform_images_parallel(input_dir, output_dir)


Function transform_image took 0.0278 secondsFunction transform_image took 0.0295 seconds

Function transform_image took 0.0422 seconds
Function transform_image took 0.0621 seconds
Function transform_image took 0.0400 seconds
Function transform_image took 0.0407 seconds
Function transform_image took 0.0579 seconds
Function transform_image took 0.0350 seconds
Function transform_image took 0.0618 secondsFunction transform_image took 0.0392 secondsFunction transform_image took 0.0509 seconds


Function transform_image took 0.0519 seconds
Function transform_image took 0.0431 seconds
Function transform_image took 0.0665 secondsFunction transform_image took 0.0663 secondsFunction transform_image took 0.0578 seconds


Function transform_image took 0.0303 secondsFunction transform_image took 0.0260 seconds

Function transform_image took 0.0512 secondsFunction transform_image took 0.0608 seconds

Function transform_image took 0.0354 seconds
Function transform_image took 0.0400 secondsFunction tr

error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'imwrite'
> Overload resolution failed:
>  - img is not a numerical tuple
>  - Expected Ptr<cv::UMat> for argument 'img'
